# Сбор и разметка данных (семинары)

## Урок 2. Парсинг HTML. BeautifulSoup

### Домашнее задание:

Выполнить скрейпинг данных в веб-сайта http://books.toscrape.com/ и извлечь информацию о всех книгах на сайте во всех категориях: название, 

цену, количество товара в наличии (In stock (19 available)) в формате integer, описание.

Затем сохранить эту информацию в JSON-файле.


In [8]:
import requests
from bs4 import BeautifulSoup
import json

# Базовый URL сайта
base_url = 'http://books.toscrape.com/'

# Получение HTML содержимого главной страницы
main_page_response = requests.get(base_url)
print(f'Основной URL статус: {main_page_response.status_code}')
main_page_soup = BeautifulSoup(main_page_response.text, 'html.parser')

# Получение всех ссылок на категории книг
category_links = [base_url + x.get('href') for x in main_page_soup.find('ul', class_='nav-list').find_all('a', href=True)[1:]]  # [1:] пропускаем первую ссылку, которая ведет на главную страницу категорий

# Словарь для хранения информации по книгам
books_info = []

# Функция для извлечения данных о книге
def extract_book_info(article):
    title = article.find('h3').find('a')['title']
    price = article.find('p', class_='price_color').text[2:]  # Убираем символ фунта
    stock_text = article.find('p', class_='instock availability').text.strip()
    stock_numbers = [int(s) for s in stock_text.split() if s.isdigit()]
    stock = stock_numbers[0] if stock_numbers else 0  # Берём первое найденное число или 0
    
    # Получаем ссылку на страницу книги для извлечения описания
    book_url = article.find('h3').find('a')['href']
    book_url = base_url + 'catalogue/' + book_url.replace('../../../', '')
    book_response = requests.get(book_url)
    print(f'Статус страницы книги {title}: {book_response.status_code}')
    book_soup = BeautifulSoup(book_response.text, 'html.parser')
    description_tag = book_soup.find('meta', attrs={'name': 'description'})
    description = description_tag['content'].strip() if description_tag else 'No description'
    
    return {
        'title': title,
        'price': float(price.replace('£', '')),
        'stock': stock,
        'description': description
    }


# Проходим по каждой категории и извлекаем информацию о книгах
for category_link in category_links:
    while True:
        response = requests.get(category_link)
        print(f'Статус категории {category_link}: {response.status_code}')
        if response.status_code != 200:
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('article', class_='product_pod')
        
        for article in articles:
            book_info = extract_book_info(article)
            books_info.append(book_info)
        
        # Проверяем, есть ли ссылка на следующую страницу в категории
        next_button = soup.find('li', class_='next')
        if next_button:
            next_page_url = next_button.find('a')['href']
            category_link = '/'.join(category_link.split('/')[:-2]) + '/' + next_page_url
        else:
            break

# Сохраняем результат в JSON-файл
with open('books_info.json', 'w', encoding='utf-8') as f:
    json.dump(books_info, f, ensure_ascii=False, indent=4)

print('Скрейпинг завершен и данные сохранены в "books_info.json"')

Основной URL статус: 200
Статус категории http://books.toscrape.com/catalogue/category/books/travel_2/index.html: 200
Статус страницы книги It's Only the Himalayas: 200
Статус страницы книги Full Moon over Noahâs Ark: An Odyssey to Mount Ararat and Beyond: 200
Статус страницы книги See America: A Celebration of Our National Parks & Treasured Sites: 200
Статус страницы книги Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel: 200
Статус страницы книги Under the Tuscan Sun: 200
Статус страницы книги A Summer In Europe: 200
Статус страницы книги The Great Railway Bazaar: 200
Статус страницы книги A Year in Provence (Provence #1): 200
Статус страницы книги The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2): 200
Статус страницы книги Neither Here nor There: Travels in Europe: 200
Статус страницы книги 1,000 Places to See Before You Die: 200
Статус категории http://books.toscrape.com/catalogue/category/books/mystery_3/index.ht